In [1]:
import chess
import chess.engine
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from chess_env import ChessEnv

2024-02-06 12:54:31.263538: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-06 12:54:31.303335: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-06 12:54:31.303377: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-06 12:54:31.304535: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-06 12:54:31.310670: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-06 12:54:31.311487: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
def build_model(input_shape, num_actions):
    model = keras.Sequential([
        layers.Input(shape=input_shape),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'),
        layers.Dense(num_actions, activation='linear')
    ])
    return model

In [3]:
from chess_env import ChessEnv

chess_env = ChessEnv()
input_shape = chess_env.observation_space.shape
print(input_shape)
analises = chess_env.engine.analyse(chess_env.board, limit=(chess.engine.Limit(time=0.1)))
print(analises['score'].relative.score(), analises.keys())

(8, 8, 6)
48 dict_keys(['string', 'depth', 'seldepth', 'multipv', 'score', 'nodes', 'nps', 'hashfull', 'tbhits', 'time', 'pv'])


In [4]:
# allows to play ramdom moves and get score
import random
pov_score = analises['score']
print("pov_score:", pov_score)
print(chess_env.board)
moves = list(chess_env.board.legal_moves)
print(moves)
chess_env.board.push(moves[random.choice(range(len(moves)))])
print(chess_env.board)


analysis = chess_env.engine.analyse(chess_env.board, limit=(chess.engine.Limit(time=0.001)))


advantage_white = analysis['score'].relative.score()
print(advantage_white)
pov_score = analises['score'].pov(chess_env.board.turn)
print("pov_score:", pov_score, "white" if chess_env.board.turn else "black", pov_score.score(), type(pov_score))
print(type(advantage_white), type(analises['score']))


pov_score: +48
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
[Move.from_uci('g1h3'), Move.from_uci('g1f3'), Move.from_uci('b1c3'), Move.from_uci('b1a3'), Move.from_uci('h2h3'), Move.from_uci('g2g3'), Move.from_uci('f2f3'), Move.from_uci('e2e3'), Move.from_uci('d2d3'), Move.from_uci('c2c3'), Move.from_uci('b2b3'), Move.from_uci('a2a3'), Move.from_uci('h2h4'), Move.from_uci('g2g4'), Move.from_uci('f2f4'), Move.from_uci('e2e4'), Move.from_uci('d2d4'), Move.from_uci('c2c4'), Move.from_uci('b2b4'), Move.from_uci('a2a4')]
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. P . . . . . .
P . P P P P P P
R N B Q K B N R
35
pov_score: -48 black -48 <class 'chess.engine.Cp'>
<class 'int'> <class 'chess.engine.PovScore'>


In [23]:
env = ChessEnv()
input_shape = (8, 8, 6)
num_actions = 4096
learning_rate = 0.001
discount_factor = 0.99
num_episodes = 100


model = build_model(input_shape, num_actions=num_actions)

for episode in range(num_episodes):
    state = env.reset()
    state = np.expand_dims(state, axis=0)

    total_reward = 0
    done = False
    while not done: 
        legal_moves = list(env.board.legal_moves)  # Implement logic to get legal moves
        if legal_moves:
            action = np.random.choice(legal_moves)
        else:
            # Handle the case when there are no legal moves
            action = 0  # Choose a default action or handle accordingly

        next_state, reward, done, info = env.step(action)
        next_state = np.expand_dims(next_state, axis=0)
        flt_reward = float(reward)
        print(flt_reward, type(flt_reward), type(discount_factor), type(reward))
       
        reward_tensor = tf.convert_to_tensor(flt_reward, dtype=tf.float32) 
        discount_tensor = tf.convert_to_tensor(-discount_factor, dtype=tf.float32)
        print(type(reward_tensor), type(discount_tensor))
        # reward_tensor = tf.cast(reward_tensor, dtype=tf.float32)
        # discount_tensor = tf.cast(discount_tensor, dtype=tf.float32)

        target = reward_tensor + discount_tensor * tf.reduce_max(next_state, axis=1)

        with tf.GradientTape() as tape:
            q_values = model(state, training=True)
            action_mask = tf.one_hot(action, num_actions)
            q_values = tf.reduce_sum(tf.multiply(q_values, action_mask), axis=1)
            loss = keras.losses.mean_squared_error(target, q_values)
        
        gradients = tape.gradient(loss, model.trainable_variables)
        model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        total_reward += reward

-18.0 <class 'float'> <class 'float'> <class 'int'>
<class 'tensorflow.python.framework.ops.EagerTensor'> <class 'tensorflow.python.framework.ops.EagerTensor'>


InvalidArgumentError: cannot compute Mul as input #1(zero-based) was expected to be a float tensor but is a int64 tensor [Op:Mul] name: 

In [9]:
input_shape = (8, 8, 6)
num_actions = 4096
learning_rate = 0.001
discount_factor = 0.99

#create DNQ model
model = build_model(input_shape, num_actions)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss='mean_squared_error'
)
env = ChessEnv()
num_episodes = 100

for episode in range(num_episodes):
    state = env.reset()
    state = np.expand_dims(state, axis=0)

    total_reward = 0
    done = False
    while not done: 
        # select actioi unsit epsilon-greedy policy
        action = np.random.choice(num_actions)
        action = tf.convert_to_tensor(float(action), dtype=tf.float32)
        next_state, reward, done, info = env.step(action)
        next_state = np.expand_dims(next_state, axis=0)
        reward = float(reward)
        reward_tensor = tf.convert_to_tensor(reward, dtype=tf.float32) 
        discount_tensor =- tf.convert_to_tensor(-discount_factor, dtype=tf.float32)

        target = reward_tensor + discount_tensor * tf.reduce_max(next_state, axis=1)

        with tf.GradientTape() as tape:
            q_values = model(state, training=True)
            action_mask = tf.one_hot(action, num_actions)
            q_values = tf.reduce_sum(tf.multiply(q_values, action_mask), axis=1)
            loss = keras.losses.mean_squared_error(target, q_values)
        
        gradients = tape.gradient(loss, model.trainable_variables)
        model.optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        total_reward += reward
        state = next_state
    print(f'Episode: {episode}, total_reward: {total_reward}')



AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'from_square'

In [ ]:
model.save('model.h5')